In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'A fascinating fact about Cristiano Ronaldo is that he has a tattoo of his late father’s name, "José Dinis Aveiro," on his left arm, symbolizing the deep bond they shared. Additionally, Ronaldo is known for his extraordinary dedication to fitness; at age 38, he was still performing at the highest levels, showcasing his remarkable longevity in professional football.'

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [6]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'A curious fact about Cristiano '

# Combining LCEL Chains

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [9]:
chain.invoke('Chamberlain')

'Warming to his role as a prominent British statesman, Neville Chamberlain is best remembered for his policy of appeasement toward Nazi Germany prior to World War II.'

# Combined chain

In [10]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [11]:
composed_chain.invoke({"politician": "Lincoln"})

'Yes, Abraham Lincoln is widely regarded as a positive and transformative figure for humanity. As the 16th President of the United States, he led the nation through its Civil War, preserving the Union, and took significant steps to abolish slavery with the issuance of the Emancipation Proclamation and his support for the 13th Amendment. His leadership helped to promote principles of equality, human rights, and national unity, making a lasting impact on the course of history and advancing the cause of justice and freedom.'

## Another example: a chain inside another chain

In [15]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Be more specific in the first prompt
prompt1 = ChatPromptTemplate.from_template(
    "What country was the politician {politician} from? Provide only the country name."
)

# Be more specific in the second prompt
prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in? Be specific and respond in {language}."
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

# Try running it again
result = chain2.invoke({"politician": "Mitterrand", "language": "English"})
print(result)

France is located in the continent of Europe.


# LCEL chain at work in a typical RAG app

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")

In [17]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\rbhar\anaconda3\envs\llmapp_latest\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition involves breaking down a complex task into smaller, more manageable steps or subgoals. Techniques like Chain of Thought and Tree of Thoughts facilitate this by guiding the model to think step-by-step or explore multiple reasoning paths. External tools, such as classical planners using PDDL, can also be employed to handle long-horizon planning by translating problems into intermediate representations.'

In [19]:
rag_chain.invoke("What is Self-Reflection?")

'Self-reflection is a process that enables agents to evaluate and improve their actions by analyzing past experiences, correcting mistakes, and refining future decisions. It involves creating higher-level summaries or inferences from memories to guide behavior and enhance reasoning skills. This iterative mechanism helps agents perform better in complex or uncertain tasks.'